In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [9]:


train_data_dir = ('Dataset/Train')
test_data_dir = ('Dataset/Test')
valid_data_dir = ('Dataset/Val')

img_width, img_height = 224, 224
batch_size = 64
input_shape = (img_width, img_height, 3)

train_datagen = ImageDataGenerator(rescale = 1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 10179 images belonging to 43 classes.
Found 1280 images belonging to 43 classes.
Found 1270 images belonging to 43 classes.


In [5]:
from tensorflow.keras.applications import InceptionV3
# Load the InceptionV3 model pre-trained on ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=10  # You can start with a small number of epochs and increase as needed
)

# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[:249]:
    layer.trainable = False
for layer in base_model.layers[249:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training the model
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=150  # Additional epochs for fine-tuning
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test accuracy: {test_accuracy:.2f}")

Epoch 1/10
 24/159 [===>..........................] - ETA: 2:00 - loss: 0.3582 - accuracy: 0.0298

d:\Software\anaconda3\envs\py310\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


159/159 [==============================] - 186s 1s/step - loss: 0.1678 - accuracy: 0.0734 - val_loss: 0.0865 - val_accuracy: 0.4359
Epoch 2/10
159/159 [==============================] - 102s 642ms/step - loss: 0.1012 - accuracy: 0.2073 - val_loss: 0.0718 - val_accuracy: 0.5370
Epoch 3/10
159/159 [==============================] - 101s 634ms/step - loss: 0.0897 - accuracy: 0.2823 - val_loss: 0.0685 - val_accuracy: 0.5880
Epoch 4/10
159/159 [==============================] - 103s 645ms/step - loss: 0.0832 - accuracy: 0.3333 - val_loss: 0.0609 - val_accuracy: 0.5905
Epoch 5/10
159/159 [==============================] - 101s 633ms/step - loss: 0.0777 - accuracy: 0.3837 - val_loss: 0.0597 - val_accuracy: 0.6118
Epoch 6/10
159/159 [==============================] - 102s 643ms/step - loss: 0.0750 - accuracy: 0.3990 - val_loss: 0.0556 - val_accuracy: 0.6340
Epoch 7/10
159/159 [==============================] - 101s 631ms/step - loss: 0.0719 - accuracy: 0.4285 - val_loss: 0.0540 - val_accuracy:

In [6]:
# Save the model
model.save('fix_model.h5')

In [8]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model('fix_model.h5')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('fix_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\brill\AppData\Local\Temp\tmpcxoaz9lc\assets


INFO:tensorflow:Assets written to: C:\Users\brill\AppData\Local\Temp\tmpcxoaz9lc\assets


In [51]:
# Load the model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
model = load_model('final_model.h5')

# Function to preprocess new image
def preprocess_image(img_path, target_size):
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# img_path = 'naskunxsate.jpg'
# img_path = 'Peluang-usaha-nasi-goreng-ayam.jpg'
# img_path = 'ayamxtest.jpg'
img_path = 'nasxayam.jpg'
processed_image = preprocess_image(img_path, target_size=(224, 224))
predictions = model.predict(processed_image)
threshold = 1e-6
predicted_labels = (predictions > threshold).astype(int)
label_map = train_generator.class_indices
label_map = dict((v,k) for k,v in label_map.items())
predicted_classes = [label_map[i] for i in range(len(predicted_labels[0])) if predicted_labels[0][i] == 1]
print(f'Predicted labels for the image: {predicted_classes}')


1/1 [==============================] - 1s 1s/step
Predicted labels for the image: ['Ayam_Crispy', 'Udang_Goreng_Tepung']
